NOTE THAT FILE LOCATIONS ARE LOCAL TO MY MACHINE AND THE FILES ARE NOT CURRENTLY ON THE GITHUB AS THEY ARE VERY LARGE

In [1]:
import mwxml
import pickle, re
import glob

In [2]:
paths = glob.glob('F:/_GroupProject/RawData/*.bz2')

In [3]:
print(len(paths))

62


In [92]:
def clean_text(text_to_clean):
    #remove text written between double curly braces
    text_to_clean = re.sub(r"{{.*}}", "",text_to_clean)
    # remove file attachments
    text_to_clean = re.sub(r"\[\[File:.*\]\]", "",text_to_clean)
    # remove image attachments
    text_to_clean = re.sub(r"\[\[Image:.*\]\]", "",text_to_clean)
    # remove headings
    text_to_clean = re.sub(r"\==.*\==", "",text_to_clean)
    text_to_clean = re.sub(r"\=.*\=", "",text_to_clean)
    # remove unwanted lines starting with special characters
    text_to_clean = re.sub(r"\n: \'\'.*", "",text_to_clean)
    text_to_clean = re.sub(r"\n!.*", "",text_to_clean)
    text_to_clean = re.sub(r"^:\'\'.*", "",text_to_clean)
    # replace non-breaking space symbols
    text_to_clean = re.sub(r"&nbsp", " ",text_to_clean)
    # remove URLs link
    text_to_clean = re.sub(r"http\S+","",text_to_clean)
    # remove digits from text (debatable, perhaps only delete numbers on their own e.g. 20 and not 20th?)
    text_to_clean = re.sub(r"\d+", "",text_to_clean)
    # remove text in brackets
    text_to_clean = re.sub(r"\(.*\)", "",text_to_clean)
    text_to_clean = re.sub(r"\<.*\>", "", text_to_clean)
    # remove sentences saying the category of the article
    text_to_clean = re.sub(r"Category:.*", "",text_to_clean)
    # remove info boxes and taxoboxes
    text_to_clean = re.sub(r"\| .*", "",text_to_clean)
    text_to_clean = re.sub(r"\n\|.*","",text_to_clean)
    text_to_clean = re.sub(r"\n \|.*","",text_to_clean)
    text_to_clean = re.sub(r".* \|\n","",text_to_clean)
    text_to_clean = re.sub(r".*\|\n","",text_to_clean)
    # remove infobox or taxobox
    text_to_clean = re.sub(r"{{Infobox.*","",text_to_clean)
    text_to_clean = re.sub(r"{{infobox.*","",text_to_clean)
    text_to_clean = re.sub(r"{{taxobox.*","",text_to_clean)
    text_to_clean = re.sub(r"{{Taxobox.*","",text_to_clean)
    text_to_clean = re.sub(r"{{ Infobox.*","",text_to_clean)
    text_to_clean = re.sub(r"{{ infobox.*","",text_to_clean)
    text_to_clean = re.sub(r"{{ taxobox.*","",text_to_clean)
    text_to_clean = re.sub(r"{{ Taxobox.*","",text_to_clean)
    # remove lines starting from *
    text_to_clean = re.sub(r"\* .*","",text_to_clean)
    # remove new line character
    text_to_clean = re.sub(r"\n","",text_to_clean)
    # remove image extensions
    text_to_clean = re.sub(r"png", "", text_to_clean)
    text_to_clean = re.sub(r"svg", "", text_to_clean)
    text_to_clean = re.sub(r"jpg", "", text_to_clean)
    # replace all punctuations with space
    text_to_clean = re.sub(r"\!|\"|\#|\$|\%|\&|\'|\(|\)|\*|\+|\,|\-|\.|\/|\:|\;|\|\?|\@|\[|\\|\]|\^|\_|\`|\{|\||\}|\~"," ",text_to_clean)
    # replace consecutive multiple space with single space
    text_to_clean = re.sub(r" +"," ",text_to_clean)
    return text_to_clean

In [101]:
def process_wikidump(dump, path):
    # iterate through pages in the dump
    for page in dump:
        # if the page does redirect then it doesn't contain any data!
        if page.redirect == None:
            x = 0
            for revision in page: # page is an iterator
                assert(x == 0) # ensure only one revision in the page (current revision)
                x += 1
                if (type(revision.text) == str): # ensure revision.text is not None
                    cleaned = clean_text(revision.text)
                    yield (page.title, clean_text(revision.text))
                
def begin_process_dump(path, output_file_name):
    current = [] # store pages
    for title, text in mwxml.map(process_wikidump, path):
        current.append((title, text)) # add to list of pages
    bf = open("F:/_GroupProject/NewWordsOnlyData/" + output_file_name, "wb")
    pickle.dump(current, bf, -1) # write the list to file
    bf.close()

In [94]:

# convert wiki xml files into text and save as binary file

for i in range(0, len(paths)):
    current_file_number = i # 1 less than number of file

    current_file = [paths[current_file_number]]
    input_file_name = re.findall("multistream\d+", current_file[0])[0]
    output_file = re.sub(r"multistream", "", input_file_name) + "_" + re.findall("p\d+p\d+", current_file[0])[0]

    begin_process_dump(current_file, output_file)


1
2
3
4


KeyboardInterrupt: 

In [1]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\nross\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [4]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import random
import pickle
import glob

In [ ]:
proc = glob.glob("F:/_GroupProject/WordsOnlyData/*")

scores = []
sid = SentimentIntensityAnalyzer()

sample_size = 0

file_counter = 0

for path in proc:
    
    print(file_counter)
    file_counter += 1
    
    bf = open(path, "rb")
    a = pickle.load(bf)

    sample = 1500
    sample_size += sample
    
    for i in range(sample):
        if (len(a[i][1]) > 100):
            score = sid.polarity_scores(a[i][1])
            scores.append(score)

    bf.close()

In [6]:
len(scores)

61908

In [8]:
negative = 0
neutral = 0
positive = 0
for i in range(len(scores)):
    # a file is category X when P(X) > 0.33
    if (scores[i]['neg'] >= scores[i]['neu'] and scores[i]['neg'] >= scores[i]['pos']):
        negative += 1
    elif (scores[i]['neu'] >= scores[i]['neg'] and scores[i]['neu'] >= scores[i]['pos']):
        neutral += 1
    else:
        positive += 1

In [9]:
assert(negative + neutral + positive == len(scores))

In [10]:
total_negative = negative / len(scores)
total_neutral = neutral / len(scores)
total_positive = positive / len(scores)

In [11]:
total_negative + total_neutral + total_positive

1.0

In [12]:
total_neutral

0.9995800219680817

In [18]:
import pickle

file = open("F:/_GroupProject/sample_data", "rb")
samples = pickle.load(file)

print(len(samples))
print("\n")
print(type(samples[0]))
print("\n")
print(samples[0][0])
print("\n")
print(samples[0][1])

49784


<class 'tuple'>


Anarchism


 Anarchism is a political philosophy and Political movement movement that is sceptical of authority and rejects all involuntary coercive forms of Social hierarchy hierarchy Anarchism calls for the abolition of the State of the socialist movement and has a strong historical association with anti capitalism and socialism Humans lived in society societies without formal hierarchies long before the establishment of formal states realm s or empire s With the rise of organised hierarchical bodies scepticism toward authority also rose Although traces of anarchist thought are found throughout history modern anarchism emerged from the Age of Enlightenment Enlightenment During the latter half of the th and the first decades of the th century the anarchist movement flourished in most parts of the world and had a significant role in workers struggles for emancipation Various anarchist schools of thought formed during this period Anarchists have taken part in s